In [1]:
import numpy as np

In [23]:
class Bandit:
    def __init__(self, k=10, exp_rate=.3):
        self.k = k
        self.actions = range(self.k)
        self.exp_rate = exp_rate
        self.total_reward = 0
        self.avg_reward = []
        
        self.TrueValue = []
        for i in range(self.k):
            self.TrueValue.append(np.random.randn())  # standard normal distribution
        
        self.values = np.zeros(self.k)
        self.times = 0
    
    def chooseAction(self):
        # explore
        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # exploit
            action = np.argmax(self.values)
        return action
    
    def takeAction(self, action):
        self.times += 1
        # take action and update value estimates
        reward = np.random.randn() + self.TrueValue[action]  # add randomness to reward
        # using incremental method to propagate
        self.values[action] += 1/self.times * (reward - self.values[action])
        
        self.total_reward += reward
        self.avg_reward.append(self.total_reward/self.times)
        
    def play(self, n):
        for _ in range(n):
            action = self.chooseAction()
            self.takeAction(action)

In [24]:
bdt = Bandit(k=5)

In [25]:
bdt.play(1000)

In [26]:
bdt.values

array([-0.77348325,  0.15234243,  0.66705359, -1.00409528, -0.17930832])

In [27]:
bdt.TrueValue

[-1.1868327454352077,
 0.7328327570619165,
 0.682876732534503,
 -1.907171534811549,
 -0.019883589615622258]

In [ ]:
sel